<a href="https://colab.research.google.com/github/J-Jaehyun-SEO/Project_Jeohui/blob/main/(3)Jeohui_1990_2024_KcELECTRA_4publish.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##install & setting

In [ ]:
import pandas as pd

In [ ]:
!pip install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu118 --quiet

In [ ]:
!pip install openpyxl adjustText --quiet

In [ ]:
# Nanum 폰트 설치
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm -rf ~/.cache/matplotlib

In [ ]:
# 필요한 라이브러리 임포트
import matplotlib.pyplot as plt
import numpy as np
import matplotlib as mpl
from matplotlib import rc

# [FONT TEST] 그래프에서 마이너스 폰트 깨지는 문제에 대한 대처
plt.rc('font', family='NanumBarunGothic')

# plot 한글 깨짐 확인용 -> 한글 깨지면 런타임 재시작 (런타임 메뉴 -> 런타임 다시 시작 후 다시 셀 실행)
data = np.random.randint(-100, 100, 50).cumsum()
plt.plot(range(50), data, 'r')
mpl.rcParams['axes.unicode_minus'] = False
plt.title('한자漢字 및 폰트 깨짐 테스트를 위한 그래프')
plt.ylabel('font test_한글/漢字')
plt.xlabel('!이 그래프의 가로 세로 축 및 범례가 표시된다면 문제 없이 표시되는 것임!')
plt.show()


In [ ]:
import torch

if torch.cuda.is_available():
  print('GPU is available!')
else:
  print('GPU is not available.')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#df = pd.read_pickle('/content/drive/Shareddrives/DHKP/2021-2023.기존연구/03. AI융합연구우리나라_저희(21년_9-12월)/[서재현] 저희/학습용데이터/new_df_0707-특수문자 무시(tokenized_merged)백업_1차.pickle')
#df = df[['jeohee_sentence']]  # Keep only the desired column
#df = df.rename(columns={'jeohee_sentence': 'sentence'})  #  Rename the column

####train

In [ ]:
train = pd.read_csv('/content/drive/Shareddrives/DHKP/2021-2023.기존연구/03. AI융합연구_우리나라_저희(21년_9-12월)/[서재현] 저희/학습용데이터/train.csv')
#train = train.rename(columns={'jeohee_sentence': 'sentence'})  # Rename the column
train = train.drop('Unnamed: 0', axis=1)
train

In [ ]:
initial_length = len(train)
filtered_df = train[train['학습용 분류'].isin([0, 1])]
removed_count = initial_length - len(filtered_df)
print("Number of rows removed:", removed_count)

####test

In [ ]:
import pandas as pd

# Assuming 'train' is your original DataFrame
test = train.sample(n=210, random_state=42)  # Extract 210 random samples
train = train.drop(test.index)  # Remove those samples from 'train'

# Reset indices for both DataFrames
train = train.reset_index(drop=True)
test = test.reset_index(drop=True)
#test = test.drop('Unnamed: 0', axis=1)


In [ ]:
test

In [ ]:
train

In [ ]:
print(train.head())
print(test.head())

# 모델 제작 Create Model

##Model

데이터셋 로드 및 필터링 함수


In [ ]:
import os
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import ElectraForSequenceClassification, AdamW, AutoTokenizer
from tqdm import tqdm
import torch.nn.functional as F
import numpy as np
import pandas as pd
import shutil

# 디바이스 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 디버깅을 위해 CUDA 런타임 블로킹 설정
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

# 데이터셋 필터링 함수
def filter_dataset(dataframe):
    return dataframe[dataframe['학습용 분류'].isin([0, 1])]

# 데이터 로드 및 필터링
train_dataframe = filter_dataset(train)
test_dataframe = filter_dataset(test)


데이터셋 클래스


In [ ]:
class ClassifyDataset(Dataset):
    def __init__(self, dataframe):
        self.dataset = dataframe.dropna(axis=0).drop_duplicates(subset=['sentence'])
        self.tokenizer = AutoTokenizer.from_pretrained("beomi/KcELECTRA-base")
        print(self.dataset.describe())

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        text = self.dataset.iloc[idx]['sentence']
        y = self.dataset.iloc[idx]['학습용 분류']
        inputs = self.tokenizer(
            text,
            return_tensors='pt',
            truncation=True,
            max_length=256,
            padding='max_length',
            add_special_tokens=True
        )
        input_ids = inputs['input_ids'][0]
        attention_mask = inputs['attention_mask'][0]
        return input_ids, attention_mask, y

# 데이터셋 생성
train_dataset = ClassifyDataset(train_dataframe)
test_dataset = ClassifyDataset(test_dataframe)


모델 및 토크나이저 설정


In [ ]:
# 모델 및 토크나이저 설정
model = ElectraForSequenceClassification.from_pretrained("beomi/KcELECTRA-base").to(device)
tokenizer = AutoTokenizer.from_pretrained("beomi/KcELECTRA-base")

# 하이퍼파라미터 설정
epochs = 5
batch_size = 16
optimizer = AdamW(model.parameters(), lr=5e-6)

# 데이터 로더 설정
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)


백업 함수


In [ ]:
# 백업 경로 설정
backup_path = '/content/drive/Shareddrives/DHKP/2021-2023.기존연구/03. AI융합연구_우리나라_저희(21년_9-12월)/[서재현] 저희/학습용데이터/'

# 백업을 위한 파일 저장 함수
def backup_data(epoch, losses, accuracies, model_state, optimizer_state):
    data = {
        'epoch': epoch,
        'losses': losses,
        'accuracies': accuracies,
        'model_state': model_state,
        'optimizer_state': optimizer_state
    }
    backup_file = f'backup_epoch_{epoch}.pth'
    torch.save(data, backup_file)
    shutil.move(backup_file, os.path.join(backup_path, backup_file))

# 모델 저장 함수
def save_model(model, save_path):
    torch.save(model.state_dict(), save_path)
    print(f"Model saved to {save_path}")

# 모델 로드 함수
def load_model(model_class, load_path, device):
    model = model_class.from_pretrained("beomi/KcELECTRA-base")
    model.load_state_dict(torch.load(load_path, map_location=device))
    model.to(device)
    model.eval()
    print(f"Model loaded from {load_path}")
    return model


In [ ]:
load_path = os.path.join(backup_path, 'saved_model_240729.pth')
load_model()

학습 루프

In [ ]:
losses = []
accuracies = []

# 학습 루프
for epoch in range(epochs):
    total_loss = 0.0
    correct = 0
    total = 0
    batches = 0

    model.train()

    for input_ids_batch, attention_masks_batch, y_batch in tqdm(train_loader):
        optimizer.zero_grad()

        input_ids_batch = input_ids_batch.to(device)
        attention_masks_batch = attention_masks_batch.to(device)
        y_batch = y_batch.to(device, dtype=torch.long)

        outputs = model(input_ids_batch, attention_mask=attention_masks_batch)
        logits = outputs.logits
        loss = F.cross_entropy(logits, y_batch)

        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        _, predicted = torch.max(logits, 1)
        correct += (predicted == y_batch).sum().item()
        total += y_batch.size(0)

        batches += 1
        if batches % 100 == 0:
            print(f"Batch {batches}: Loss {total_loss / batches:.4f}, Accuracy {correct / total:.4f}")

    losses.append(total_loss / batches)
    accuracies.append(correct / total)
    print(f"Epoch {epoch + 1}: Train Loss {total_loss / batches:.4f}, Accuracy {correct / total:.4f}")

    # 백업 데이터 저장
    backup_data(epoch, losses, accuracies, model.state_dict(), optimizer.state_dict())

# 학습 완료 후 모델 저장
model_save_path = os.path.join(backup_path, 'saved_model.pth')
save_model(model, model_save_path)

print("Training complete.")


평가 및 예측 저장


In [ ]:
# 개별 문장 예측 함수에서 얻은 예측 결과 저장
def save_predictions(test_pd, predictions, accuracy):
    test_pd['predicted_label'] = predictions
    result_file = os.path.join(backup_path, 'predicted_results.csv')
    test_pd.to_csv(result_file, index=False)
    print(f"Predictions and accuracy saved to {result_file}")
    with open(os.path.join(backup_path, 'accuracy.txt'), 'w') as f:
        f.write(f"Test Accuracy: {accuracy}\n")

# 모델 평가 및 예측 결과 저장
def evaluate_and_save_predictions(model, test_loader, test_pd, backup_path):
    model.eval()
    test_correct = 0
    test_total = 0
    predictions = []

    with torch.no_grad():
        for input_ids_batch, attention_masks_batch, y_batch in tqdm(test_loader):
            input_ids_batch = input_ids_batch.to(device)
            attention_masks_batch = attention_masks_batch.to(device)
            y_batch = y_batch.to(device, dtype=torch.long)

            outputs = model(input_ids_batch, attention_mask=attention_masks_batch)
            logits = outputs.logits
            _, predicted = torch.max(logits, 1)
            test_correct += (predicted == y_batch).sum().item()
            test_total += y_batch.size(0)
            predictions.extend(predicted.cpu().numpy())

    accuracy = test_correct / test_total
    print("Test Accuracy:", accuracy)
    save_predictions(test_pd, predictions, accuracy)

# 모델 평가 및 예측 결과 저장
evaluate_and_save_predictions(model, test_loader, test_dataframe, backup_path)


개별 문장 예측


In [ ]:
# 개별 문장 예측 함수
def sentences_predict(sent):
    tokenized_sent = tokenizer(
        sent,
        return_tensors="pt",
        truncation=True,
        add_special_tokens=True,
        max_length=256
    )
    tokenized_sent = {key: value.to(device) for key, value in tokenized_sent.items()}

    with torch.no_grad():
        outputs = model(
            input_ids=tokenized_sent['input_ids'],
            attention_mask=tokenized_sent['attention_mask'],
            token_type_ids=tokenized_sent.get('token_type_ids')
        )

    logits = outputs[0]
    logits = logits.detach().cpu().numpy()
    result = np.argmax(logits)
    return result

# 개별 문장 예측
test_pd = filter_dataset(test_dataframe)  # 데이터 필터링

score = []
total_len = len(test_pd)

# tqdm을 사용하여 간결하게 진행 상황 표시
for review in tqdm(test_pd['sentence'], desc="Predicting sentences"):
    pred = sentences_predict(review)  # 예측 결과
    score.append(pred)

# 예측 결과 저장
save_predictions(test_pd, score)


##모델 평가

In [ ]:
import os
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import ElectraForSequenceClassification, AdamW, AutoTokenizer
from tqdm import tqdm
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Set the CUDA launch blocking (useful for debugging CUDA errors)
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

# Dataset filtering function
def filter_dataset(dataframe):
    return dataframe[dataframe['학습용 분류'].isin([0, 1])]

# Load the dataset
def load_data(train_df, test_df):
    train_dataframe = filter_dataset(train_df)
    test_dataframe = filter_dataset(test_df)
    return train_dataframe, test_dataframe

# Define Dataset Class
class ClassifyDataset(Dataset):
    def __init__(self, dataframe):
        self.dataset = dataframe.dropna(axis=0).drop_duplicates(subset=['sentence'])
        self.tokenizer = AutoTokenizer.from_pretrained("beomi/KcELECTRA-base")
        print(self.dataset.describe())

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        text = self.dataset.iloc[idx]['sentence']
        y = self.dataset.iloc[idx]['학습용 분류']
        inputs = self.tokenizer(
            text,
            return_tensors='pt',
            truncation=True,
            max_length=256,
            padding='max_length',
            add_special_tokens=True
        )
        input_ids = inputs['input_ids'][0]
        attention_mask = inputs['attention_mask'][0]
        return input_ids, attention_mask, y

# Load the model
def load_model(model_class, load_path, device):
    model = model_class.from_pretrained("beomi/KcELECTRA-base", num_labels=2)  # Assuming binary classification
    model.load_state_dict(torch.load(load_path, map_location=device))
    model.to(device)
    model.eval()
    print(f"Model loaded from {load_path}")
    return model

# Model prediction function for individual sentences
def sentences_predict(sent, model, tokenizer, device):
    tokenized_sent = tokenizer(
        sent,
        return_tensors="pt",
        truncation=True,
        add_special_tokens=True,
        max_length=256
    )
    tokenized_sent = {key: value.to(device) for key, value in tokenized_sent.items()}

    with torch.no_grad():
        outputs = model(
            input_ids=tokenized_sent['input_ids'],
            attention_mask=tokenized_sent['attention_mask'],
            token_type_ids=tokenized_sent.get('token_type_ids')
        )

    logits = outputs.logits.detach().cpu().numpy()
    result = np.argmax(logits)
    return result

# Function to evaluate the model using test dataset and calculate metrics
def evaluate_model(model, test_loader):
    model.eval()
    predictions = []
    true_labels = []

    with torch.no_grad():
        for input_ids_batch, attention_masks_batch, y_batch in tqdm(test_loader):
            input_ids_batch = input_ids_batch.to(device)
            attention_masks_batch = attention_masks_batch.to(device)
            y_batch = y_batch.to(device, dtype=torch.long)

            outputs = model(input_ids_batch, attention_mask=attention_masks_batch)
            logits = outputs.logits
            _, predicted = torch.max(logits, 1)

            predictions.extend(predicted.cpu().numpy())
            true_labels.extend(y_batch.cpu().numpy())

    # Calculate metrics
    accuracy = accuracy_score(true_labels, predictions)
    precision = precision_score(true_labels, predictions, average='binary')
    recall = recall_score(true_labels, predictions, average='binary')
    f1 = f1_score(true_labels, predictions, average='binary')

    print(f"Accuracy: {accuracy}")
    print(f"Precision: {precision}")
    print(f"Recall: {recall}")
    print(f"F1 Score: {f1}")

    return accuracy, precision, recall, f1, predictions

# Save the predictions and metrics
def save_results(test_dataframe, predictions, backup_path):
    test_dataframe['predicted_label'] = predictions
    result_file = os.path.join(backup_path, 'predicted_results.csv')
    test_dataframe.to_csv(result_file, index=False)
    print(f"Results saved to {result_file}")

# Main function to run the process
def main(train_df, test_df, backup_path):
    # Load and filter datasets
    train_dataframe, test_dataframe = load_data(train_df, test_df)

    # Create datasets
    train_dataset = ClassifyDataset(train_dataframe)
    test_dataset = ClassifyDataset(test_dataframe)

    # Create data loaders
    batch_size = 16
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

    # Load the model
    model_save_path = os.path.join(backup_path, 'saved_model_240729.pth')
    model = load_model(ElectraForSequenceClassification, model_save_path, device)

    # Evaluate the model
    accuracy, precision, recall, f1, predictions = evaluate_model(model, test_loader)

    # Save the results
    save_results(test_dataframe, predictions, backup_path)



In [ ]:
import os
import pandas as pd

# Backup path 설정
backup_path = '/content/drive/Shareddrives/DHKP/2021-2023.기존연구/03. AI융합연구_우리나라_저희(21년_9-12월)/[서재현] 저희/학습용데이터/'

# train_df와 test_df 파일 경로를 backup_path와 결합
train_file_path = os.path.join(backup_path, 'train_240729.csv')  # 학습용 CSV 파일
test_file_path = os.path.join(backup_path, 'test_240729.csv')    # 테스트용 CSV 파일

# 데이터 로드
train_df = pd.read_csv(train_file_path)
test_df = pd.read_csv(test_file_path)


In [ ]:

# main 함수 실행
main(train_df, test_df, backup_path)

In [ ]:
print(f"Original test dataframe length: {len(test_dataframe)}")
print(f"Predictions length: {len(predictions)}")


In [ ]:
test = pd.read_csv('/content/drive/Shareddrives/DHKP/2021-2023.기존연구/03. AI융합연구_우리나라_저희(21년_9-12월)/[서재현] 저희/학습용데이터/test_240729.csv')

In [ ]:
test = test.drop('Unnamed: 0', axis=1)
test

In [ ]:
def evaluate_model(model, test_loader):
    model.eval()
    predictions = []
    true_labels = []

    with torch.no_grad():
        for input_ids_batch, attention_masks_batch, y_batch in tqdm(test_loader):
            input_ids_batch = input_ids_batch.to(device)
            attention_masks_batch = attention_masks_batch.to(device)
            y_batch = y_batch.to(device, dtype=torch.long)

            # Debug: Check input data
            print(f"input_ids_batch: {input_ids_batch.shape}")
            print(f"attention_masks_batch: {attention_masks_batch.shape}")
            print(f"y_batch: {y_batch.shape}")

            outputs = model(input_ids_batch, attention_mask=attention_masks_batch)

            logits = outputs.logits
            _, predicted = torch.max(logits, 1)

            predictions.extend(predicted.cpu().numpy())
            true_labels.extend(y_batch.cpu().numpy())

    # Calculate metrics
    accuracy = accuracy_score(true_labels, predictions)
    precision = precision_score(true_labels, predictions, average='binary')
    recall = recall_score(true_labels, predictions, average='binary')
    f1 = f1_score(true_labels, predictions, average='binary')

    print(f"Accuracy: {accuracy}")
    print(f"Precision: {precision}")
    print(f"Recall: {recall}")
    print(f"F1 Score: {f1}")

    return accuracy, precision, recall, f1, predictions


In [ ]:
evaluate_model()

In [ ]:
def load_model(model_class, load_path, device):
    model = model_class.from_pretrained("beomi/KcELECTRA-base", num_labels=2)  # Assuming binary classification
    model.load_state_dict(torch.load(load_path, map_location=device))
    model.to(device)
    model.eval()

    # Debug: Ensure model is loaded
    print(f"Model loaded from {load_path}")
    return model


In [ ]:
# Define your paths and run the main function
backup_path = '/content/drive/Shareddrives/DHKP/2021-2023.기존연구/03. AI융합연구_우리나라_저희(21년_9-12월)/[서재현] 저희/학습용데이터/'
train_df = train # Update with actual path
test_df = test_pd    # Update with actual path

main(train_df, test_pd, backup_path)

In [ ]:
test

In [ ]:
train

## Learn

In [ ]:
epochs = 5
batch_size = 16

In [ ]:
epochs = 5
batch_size = 16
optimizer = AdamW(model.parameters(), lr=5e-6)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=True)

In [ ]:
# 모델 저장하기
torch.save(model.state_dict(), "jh_model.pt")

In [ ]:
import pandas as pd
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import ElectraForSequenceClassification, AdamW, AutoTokenizer
from tqdm import tqdm
import torch.nn.functional as F

# 디바이스 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 디버깅을 위해 CUDA 런타임 블로킹 설정
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

# 데이터셋 필터링 함수
def filter_dataset(dataframe):
    return dataframe[dataframe['학습용 분류'].isin([0, 1])]

# 데이터 로드 및 필터링
train_dataframe = filter_dataset(train)
test_dataframe = filter_dataset(test)

class ClassifyDataset(Dataset):
    def __init__(self, dataframe):
        self.dataset = dataframe.dropna(axis=0).drop_duplicates(subset=['sentence'])
        self.tokenizer = AutoTokenizer.from_pretrained("beomi/KcELECTRA-base")
        print(self.dataset.describe())

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        text = self.dataset.iloc[idx]['sentence']
        y = self.dataset.iloc[idx]['학습용 분류']
        inputs = self.tokenizer(
            text,
            return_tensors='pt',
            truncation=True,
            max_length=256,
            padding='max_length',
            add_special_tokens=True
        )
        input_ids = inputs['input_ids'][0]
        attention_mask = inputs['attention_mask'][0]
        return input_ids, attention_mask, y

# 데이터셋 생성
train_dataset = ClassifyDataset(train_dataframe)
test_dataset = ClassifyDataset(test_dataframe)

# 모델 및 토크나이저 설정
model = ElectraForSequenceClassification.from_pretrained("beomi/KcELECTRA-base").to(device)
tokenizer = AutoTokenizer.from_pretrained("beomi/KcELECTRA-base")

# 하이퍼파라미터 설정
epochs = 5
batch_size = 16
optimizer = AdamW(model.parameters(), lr=5e-6)

# 데이터 로더 설정
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

losses = []
accuracies = []

# 학습 루프
for epoch in range(epochs):
    total_loss = 0.0
    correct = 0
    total = 0
    batches = 0

    model.train()

    for input_ids_batch, attention_masks_batch, y_batch in tqdm(train_loader):
        optimizer.zero_grad()

        input_ids_batch = input_ids_batch.to(device)
        attention_masks_batch = attention_masks_batch.to(device)
        y_batch = y_batch.to(device, dtype=torch.long)

        outputs = model(input_ids_batch, attention_mask=attention_masks_batch)
        logits = outputs.logits
        loss = F.cross_entropy(logits, y_batch)

        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        _, predicted = torch.max(logits, 1)
        correct += (predicted == y_batch).sum().item()
        total += y_batch.size(0)

        batches += 1
        if batches % 100 == 0:
            print(f"Batch {batches}: Loss {total_loss / batches:.4f}, Accuracy {correct / total:.4f}")

    losses.append(total_loss / batches)
    accuracies.append(correct / total)
    print(f"Epoch {epoch + 1}: Train Loss {total_loss / batches:.4f}, Accuracy {correct / total:.4f}")

print("Training complete.")


테스트 데이터셋 정확도 확인하기

In [ ]:
# 모델 평가
model.eval()

test_correct = 0
test_total = 0

with torch.no_grad():
    for input_ids_batch, attention_masks_batch, y_batch in tqdm(test_loader):
        input_ids_batch = input_ids_batch.to(device)
        attention_masks_batch = attention_masks_batch.to(device)
        y_batch = y_batch.to(device, dtype=torch.long)

        outputs = model(input_ids_batch, attention_mask=attention_masks_batch)
        logits = outputs.logits
        _, predicted = torch.max(logits, 1)
        test_correct += (predicted == y_batch).sum().item()
        test_total += y_batch.size(0)

print("Test Accuracy:", test_correct / test_total)

In [ ]:
backup_path = '/content/drive/Shareddrives/DHKP/2021-2023.기존연구/03. AI융합연구_우리나라_저희(21년_9-12월)/[서재현] 저희/학습용데이터'

In [ ]:
import pandas as pd

# 개별 문장 예측 함수에서 얻은 예측 결과 저장
def save_predictions(test_pd, predictions):
    test_pd['predicted_label'] = predictions
    result_file = os.path.join(backup_path, 'predicted_results.csv')
    test_pd.to_csv(result_file, index=False)

In [ ]:
from tqdm import tqdm

predictions = []

# 개별 문장 예측
test_pd = filter_dataset(test_dataframe)  # 데이터 필터링

score = []
total_len = len(test_pd)

# tqdm을 사용하여 간결하게 진행 상황 표시
for review in tqdm(test_pd['sentence'], desc="Predicting sentences"):
    pred = sentences_predict(review)  # 예측 결과
    score.append(pred)

# 예측 결과 저장
save_predictions(test_pd, score)

# 예측 결과 출력
test_pd['predicted_label'] = score
print(test_pd[['sentence', 'predicted_label']])

In [ ]:
test_pd

In [ ]:
test_pd['score'] = 0
test_pd['predicted'] =0

In [ ]:
len(score)

In [ ]:
from tqdm import tqdm

# 개별 문장 예측 함수
def sentences_predict(sent):
    tokenizer = AutoTokenizer.from_pretrained("beomi/KcELECTRA-base")
    model.eval()
    tokenized_sent = tokenizer(
        sent,
        return_tensors="pt",
        truncation=True,
        add_special_tokens=True,
        max_length=256
    )
    tokenized_sent = {key: value.to(device) for key, value in tokenized_sent.items()}

    with torch.no_grad():
        outputs = model(
            input_ids=tokenized_sent['input_ids'],
            attention_mask=tokenized_sent['attention_mask'],
            token_type_ids=tokenized_sent.get('token_type_ids')
        )

    logits = outputs[0]
    logits = logits.detach().cpu().numpy()
    result = np.argmax(logits)
    return result

# 개별 문장 예측
test_pd = filter_dataset(test_dataframe)  # 데이터 필터링

score = []
total_len = len(test_pd)

# tqdm을 사용하여 간결하게 진행 상황 표시
for review in tqdm(test_pd['sentence'], desc="Predicting sentences"):
    pred = sentences_predict(review)  # 예측 결과
    score.append(pred)

# 예측 결과와 데이터프레임 길이 확인 및 처리
if len(score) != len(test_pd):
    raise ValueError(f"Length of predictions ({len(score)}) does not match length of index ({len(test_pd)})")

# 예측 결과 저장
def save_predictions(test_pd, predictions):
    test_pd['predicted_label'] = predictions
    result_file = os.path.join(backup_path, 'predicted_results.csv')
    test_pd.to_csv(result_file, index=False)

save_predictions(test_pd, score)


In [ ]:
test_pd

In [ ]:
!pip install openpyxl

In [ ]:
#test_pd.to_csv('저희 모델 분류 결과_withscore.csv', index=False)
test_pd.to_excel('저희 모델 분류 결과_withscore.xlsx', index=False)

In [ ]:
import torch

# 모델 저장 경로
model_save_path = '/content/drive/Shareddrives/DHKP/2021-2023.기존연구/03. AI융합연구_우리나라_저희(21년_9-12월)/[서재현] 저희/학습용데이터/saved_model.pth'

# 모델 저장 함수
def save_model(model, save_path):
    torch.save(model.state_dict(), save_path)
    print(f"Model saved to {save_path}")

# 학습이 완료된 후 모델 저장
save_model(model, model_save_path)


In [ ]:
# GPU 사용
device = torch.device("cuda")

#전체 데이터 분류

In [ ]:
import os
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import ElectraForSequenceClassification, AutoTokenizer
from tqdm import tqdm
import numpy as np
import pandas as pd

# 디바이스 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 백업 경로 설정
backup_path = '/content/drive/Shareddrives/DHKP/2021-2023.기존연구/03. AI융합연구_우리나라_저희(21년_9-12월)/[서재현] 저희/학습용데이터/'
model_save_path = os.path.join(backup_path, 'saved_model_240729.pth')

In [ ]:
# 모델 및 토크나이저 설정
model = ElectraForSequenceClassification.from_pretrained("beomi/KcELECTRA-base").to(device)
tokenizer = AutoTokenizer.from_pretrained("beomi/KcELECTRA-base")

# 모델 로드 함수
def load_model(model_class, load_path, device):
    model = model_class.from_pretrained("beomi/KcELECTRA-base")
    model.load_state_dict(torch.load(load_path, map_location=device))
    model.to(device)
    model.eval()
    print(f"Model loaded from {load_path}")
    return model

# 모델 로드
model = load_model(ElectraForSequenceClassification, model_save_path, device)

In [ ]:
# 데이터셋 클래스 정의
class ClassifyDataset(Dataset):
    def __init__(self, dataframe):
        self.dataset = dataframe
        self.tokenizer = AutoTokenizer.from_pretrained("beomi/KcELECTRA-base")

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        text = self.dataset.iloc[idx]['sentence']
        inputs = self.tokenizer(
            text,
            return_tensors='pt',
            truncation=True,
            max_length=256,
            padding='max_length',
            add_special_tokens=True
        )
        input_ids = inputs['input_ids'][0]
        attention_mask = inputs['attention_mask'][0]
        return input_ids, attention_mask

####test - 문장 분류 잘함.

In [ ]:
#문장 하나하나 분류
def sentences_predict(sent):
    tokenizer = AutoTokenizer.from_pretrained("beomi/KcELECTRA-base")
    model.eval()
    tokenized_sent = tokenizer(
            sent,
            return_tensors="pt",
            truncation=True,
            add_special_tokens=True,
            max_length=256
    )
    tokenized_sent.to(device)

    with torch.no_grad():# 그라디엔트 계산 비활성화
        outputs = model(
            input_ids=tokenized_sent['input_ids'],
            attention_mask=tokenized_sent['attention_mask'],
            token_type_ids=tokenized_sent['token_type_ids']
            )

    logits = outputs[0]
    logits = logits.detach().cpu().numpy()
    result = np.argmax(logits)
    return result

In [ ]:
sentences_predict('저희 쪽에서도 다 생각이 있겠지?')

In [ ]:
sentences_predict('도대체 어쩌란 말입니까... 저희 잘못인 것은 저희도 알고 있습니다.')

In [ ]:
sentences_predict('도대체 어쩌란 말이냐! 저희로 썩 꺼져라')

In [ ]:
sentences_predict('도대체 어쩌란 말이냐! "저희들이 알아서 하겠나이다"')

In [ ]:
sentences_predict('넌 너가 무엇을 분류하는지 알고 있느냐? 그건 저희들이 알아서 무얼하려고 물어보는지..')

###분류를 위한 전체 데이터 로드

In [ ]:
third_data = pd.read_pickle('/content/drive/Shareddrives/DHKP/2021-2023.기존연구/03. AI융합연구_우리나라_저희(21년_9-12월)/[서재현] 저희/학습용데이터/new_df_0707-처리완(특수기호_tokenized_merged)백업_2차.pickle')
#third_data = third_data[['jeohee_sentence']]  # Keep only the desired column
third_data = third_data.rename(columns={'jeohee_sentence': 'sentence'})  #  Rename the column
#df['학습용 분류'] = 0

제3의 데이터셋 생성

In [ ]:
# 제3의 데이터셋 생성
third_dataset = ClassifyDataset(third_data)
third_loader = DataLoader(third_dataset, batch_size=16, shuffle=False)

In [ ]:
# 개별 문장 예측 함수
def sentences_predict(input_ids, attention_mask):
    input_ids = input_ids.to(device)
    attention_mask = attention_mask.to(device)

    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)

    logits = outputs.logits
    probabilities = torch.nn.functional.softmax(logits, dim=-1)
    predicted_labels = torch.argmax(probabilities, dim=1)
    return predicted_labels.cpu().numpy(), probabilities.cpu().numpy(), logits.cpu().numpy()

predictions = []
probabilities_list = []
logits_list = []

# tqdm을 사용하여 간결하게 진행 상황 표시
for input_ids_batch, attention_masks_batch in tqdm(third_loader, desc="Predicting third data"):
    preds, probs, logits = sentences_predict(input_ids_batch, attention_masks_batch)
    predictions.extend(preds)
    probabilities_list.extend(probs)
    logits_list.extend(logits)

# 예측 결과를 원본 데이터프레임에 추가
if len(predictions) == len(third_data):
    third_data['predicted_label'] = predictions
    if probabilities_list and logits_list:  # 리스트가 비어있지 않은지 확인
        for i in range(probabilities_list[0].shape[0]):
            third_data[f'prob_class_{i}'] = [prob[i] for prob in probabilities_list]
        for i in range(logits_list[0].shape[0]):
            third_data[f'logit_class_{i}'] = [logit[i] for logit in logits_list]
else:
    print(f"Warning: Length of predictions ({len(predictions)}) does not match length of data ({len(third_data)})")

# 예측 결과 저장
third_data.to_csv(os.path.join(backup_path, 'v2_040729_third_data_predictions.csv'), index=False)
print(f"Predictions saved to {os.path.join(backup_path, 'v2_040729_third_data_predictions.csv')}")

데이터셋의 총 샘플 수가 31,967개이고, 배치 크기가 16인 경우: DataLoader는 31,967개의 샘플을 1,998개의 배치로 나눕니다. 각 배치는 16개의 샘플을 포함하며, 마지막 배치는 15개의 샘플을 포함합니다.1,998개의 배치에 대한 진행 상황을 표시

#data analysis

v3_jeohui_040730_third_data_predictions.pickle

In [ ]:
import os
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import ElectraForSequenceClassification, AutoTokenizer
from tqdm import tqdm
import numpy as np
import pandas as pd

In [ ]:
predictions_df = pd.read_pickle('/content/drive/Shareddrives/DHKP/2021-2023.기존연구/03. AI융합연구_우리나라_저희(21년_9-12월)/[서재현] 저희/학습용데이터/v3_jeohui_040730_third_data_predictions.pickle')

In [ ]:
predictions_df

##기술통계량

In [ ]:
# Convert the year into decades (including up to 2024)
predictions_df['decade'] = pd.cut(predictions_df['year'], bins=[1950, 1960, 1970, 1980, 1990, 2000, 2010, 2020, 2025],
                      labels=["50s", "60s", "70s", "80s", "90s", "2000s", "2010s", "2020s"])

# Group by decade and predicted_label to calculate frequency
grouped = predictions_df.groupby(['decade', 'predicted_label']).size().unstack(fill_value=0)

# Calculate total for each decade
grouped['Total'] = grouped.sum(axis=1)

# Calculate percentages for '겸양의 저희' (0) and '저편의 저희' (1)
grouped['% 저편'] = (grouped.get(0, 0) / grouped['Total'] * 100).round(2)
grouped['% 겸양'] = (grouped.get(1, 0) / grouped['Total'] * 100).round(2)

# Add a grand total row for the sum of all decades
grand_total = pd.DataFrame(grouped.sum()).transpose()
grand_total.index = ['총합']

# Concatenate the grand total row to the grouped data
grouped = pd.concat([grouped, grand_total])

# Display or save the final table
grouped

In [ ]:
#predictions_df.to_pickle('/content/drive/Shareddrives/DHKP/2021-2023.기존연구/03. AI융합연구_우리나라_저희(21년_9-12월)/[서재현] 저희/학습용데이터/v3_jeohui_040730_third_data_predictions.pickle')

In [ ]:

# Data based on the provided values for the table
data = {
    "저편의 저희": ["488(47.10%)", "505(29.99%)", "499(25.92%)", "466(18.40%)", "334(10.28%)", "140(8.51%)", "82(0.91%)", "68(0.58%)", "2582(21%)"],
    "겸양의 저희": ["548(52.90%)", "1179(70.01%)", "1426(74.08%)", "2067(81.60%)", "2916(89.72%)", "1506(91.49%)", "8976(99.09%)", "11603(99.42%)", "30221(79%)"],
    "전체 저희": [1036, 1684, 1925, 2533, 3250, 1646, 9058, 11671, 32803]
}

# Create the index for the years
index = ["50s", "60s", "70s", "80s", "90s", "2000s", "2010s", "2020s", "총합"]

# Create the dataframe
df = pd.DataFrame(data, index=index)

# Plot the table using Matplotlib
fig, ax = plt.subplots(figsize=(10, 4))  # set the figure size
ax.axis('tight')
ax.axis('off')

# Create table
table = ax.table(cellText=df.values, colLabels=df.columns, rowLabels=df.index, loc='center', cellLoc='center')

# Style the table (optional but helpful to match your format)
table.auto_set_font_size(False)
table.set_fontsize(12)
table.scale(1.2, 1.2)

# Display the table
plt.show()

"겸양의 저희"_높은 신뢰도 문장 필터링

In [ ]:
# 클래스 1에 대한 확률이 0.9 이상인 문장 필터링
high_confidence_class_1 = predictions_df[predictions_df['prob_class_1'] >= 0.9]

# 결과 확인
print("High Confidence Predictions:")
print(high_confidence_class_1.head())


In [ ]:
high_confidence_class_1.to_excel('high_confidence_class_1.xlsx')

In [ ]:
high_confidence_class_1

"저편의 저희" 신뢰도 높은 문장 필터링

In [ ]:
# 클래스 0에 대한 확률이 0.9 이상인 문장 필터링
#high_confidence_class_0 = predictions_df[predictions_df['prob_class_0'] >= predictions_df['prob_class_1']]
high_confidence_class_0 = predictions_df[predictions_df['prob_class_0'] >= 0.8]

# 결과 확인
print("High Confidence Class 0 Predictions:")
print(high_confidence_class_0.head())


In [ ]:
high_confidence_class_0.to_excel('high_confidence_class_0.xlsx')

In [ ]:
high_confidence_class_0

예측이 불확실한 문장 필터링


In [ ]:
# 클래스 0과 1에 대한 확률 차이가 0.1 이하인 문장 필터링
uncertain_predictions = predictions_df[abs(predictions_df['prob_class_0'] - predictions_df['prob_class_1']) <= 0.2]

# 결과 확인
print("Uncertain Predictions:")
print(uncertain_predictions.head())


In [ ]:
uncertain_predictions.to_excel('uncerain_predictions.xlsx')

클래스 1로 예측된 문장들 추출 및 통계 정보 확인


In [ ]:
# 클래스 1로 예측된 문장들 추출
class_1_predictions = predictions_df[predictions_df['predicted_label'] == 1]

# 추가 작업 예시: 클래스 1로 예측된 문장들의 통계 정보 확인
print("Class 1 Predictions Statistics:")
print(class_1_predictions.describe())


In [ ]:
class_1_predictions.to_excel('class_1_predictions.xlsx')

In [ ]:
# 클래스 0으로 예측된 문장들 추출
class_0_predictions = predictions_df[predictions_df['predicted_label'] == 0]

# 추가 작업 예시: 클래스 0으로 예측된 문장들의 통계 정보 확인
print("Class 0 Predictions Statistics:")
print(class_0_predictions.describe())


In [ ]:
class_0_predictions.to_excel('class_0_predictions.xlsx')

혼동 행렬 및 분류 보고서 생성


실제 레이블이 있는 경우 혼동 행렬을 계산하여 모델의 성능을 평가할 수 있습니다.



In [ ]:
test

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

# 실제 레이블과 예측 레이블이 모두 있는 경우
if 'actual_label' in test.columns:
    y_true = predictions_df['actual_label']
    y_pred = predictions_df['predicted_label']

    # 혼동 행렬 계산
    conf_matrix = confusion_matrix(y_true, y_pred)
    print("Confusion Matrix:")
    print(conf_matrix)

    # 분류 보고서 생성
    class_report = classification_report(y_true, y_pred)
    print("Classification Report:")
    print(class_report)
else:
    print("Actual labels are not available in the dataset.")


In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

# 실제 레이블과 예측 레이블이 모두 있는 경우
if 'actual_label' in predictions_df.columns:
    y_true = predictions_df['actual_label']
    y_pred = predictions_df['predicted_label']

    # 혼동 행렬 계산
    conf_matrix = confusion_matrix(y_true, y_pred)
    print("Confusion Matrix:")
    print(conf_matrix)

    # 분류 보고서 생성
    class_report = classification_report(y_true, y_pred)
    print("Classification Report:")
    print(class_report)
else:
    print("Actual labels are not available in the dataset.")


# Visualization

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from adjustText import adjust_text

# 연도별 데이터 프레임 생성 함수
def create_yearly_df(df):
    yearly_df = df.groupby(['year', 'predicted_label']).size().unstack(fill_value=0)
    yearly_df.columns = ['지칭의 저희', '겸양의 저희']
    yearly_df.reset_index(inplace=True)
    return yearly_df

# '겸양의 저희'와 '지칭의 저희' 연도별 빈도 추이 그래프 그리기
def plot_trend(yearly_df):
    plt.figure(figsize=(12, 6))

    # 원 데이터 플롯
    plt.plot(yearly_df['year'], yearly_df['겸양의 저희'], marker='o', linestyle='-', color='b', label='겸양의 저희')
    plt.plot(yearly_df['year'], yearly_df['지칭의 저희'], marker='o', linestyle='-', color='r', label='지칭의 저희')

    # '겸양의 저희' 추세선 추가
    z1 = np.polyfit(yearly_df['year'], yearly_df['겸양의 저희'], 1)
    p1 = np.poly1d(z1)
    plt.plot(yearly_df['year'], p1(yearly_df['year']), linestyle='--', color='b', alpha=0.5)

    # '지칭의 저희' 추세선 추가
    z2 = np.polyfit(yearly_df['year'], yearly_df['지칭의 저희'], 1)
    p2 = np.poly1d(z2)
    plt.plot(yearly_df['year'], p2(yearly_df['year']), linestyle='--', color='r', alpha=0.5)

    plt.title('신문기사 연도별 저희 추세 그래프')
    plt.xlabel('연도')
    plt.ylabel('빈도수')
    plt.grid(True)

    texts = []
    for i, row in yearly_df.iterrows():
        texts.append(plt.text(row['year'], row['겸양의 저희'], row['겸양의 저희'], ha='center', va='bottom', fontsize=6.5, color='black'))
        texts.append(plt.text(row['year'], row['지칭의 저희'], row['지칭의 저희'], ha='center', va='top', fontsize=6.5, color='black'))

    adjust_text(texts, arrowprops=dict(arrowstyle="->", color='gray', lw=0.5))

    plt.legend(loc='upper left')
    plt.xticks(yearly_df['year'], rotation=90)
    plt.show()

# 연도별 데이터 프레임 생성
yearly_df = create_yearly_df(predictions_df)

# 연도별 추이 그래프 그리기
plot_trend(yearly_df)


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from adjustText import adjust_text

# 연대별 데이터 프레임 생성 함수
def create_decade_df(df):
    df['decade'] = (df['year'] // 10) * 10
    decade_df = df.groupby(['decade', 'predicted_label']).size().unstack(fill_value=0)
    decade_df.columns = ['지칭의 저희', '겸양의 저희']
    decade_df.reset_index(inplace=True)
    return decade_df

# '겸양의 저희'와 '지칭의 저희' 연대별 빈도 추이 그래프 그리기
def plot_trend(decade_df):
    plt.figure(figsize=(12, 6))

    # 원 데이터 플롯
    plt.plot(decade_df['decade'], decade_df['겸양의 저희'], marker='o', linestyle='-', color='b', label='겸양의 저희')
    plt.plot(decade_df['decade'], decade_df['지칭의 저희'], marker='o', linestyle='-', color='r', label='지칭의 저희')

    # '겸양의 저희' 추세선 추가
    z1 = np.polyfit(decade_df['decade'], decade_df['겸양의 저희'], 1)
    p1 = np.poly1d(z1)
    plt.plot(decade_df['decade'], p1(decade_df['decade']), linestyle='--', color='b', alpha=0.5)

    # '지칭의 저희' 추세선 추가
    z2 = np.polyfit(decade_df['decade'], decade_df['지칭의 저희'], 1)
    p2 = np.poly1d(z2)
    plt.plot(decade_df['decade'], p2(decade_df['decade']), linestyle='--', color='r', alpha=0.5)

    plt.title('신문기사 연대별 저희 추세 그래프')
    plt.xlabel('연대')
    plt.ylabel('빈도수')
    plt.grid(True)

    texts = []
    for i, row in decade_df.iterrows():
        texts.append(plt.text(row['decade'], row['겸양의 저희'], row['겸양의 저희'], ha='center', va='bottom', fontsize=9, color='black'))
        texts.append(plt.text(row['decade'], row['지칭의 저희'], row['지칭의 저희'], ha='center', va='top', fontsize=9, color='black'))

    adjust_text(texts, arrowprops=dict(arrowstyle="->", color='gray', lw=0.5))

    plt.legend(loc='upper left')
    plt.xticks(decade_df['decade'])
    plt.show()

# 연대별 데이터 프레임 생성
decade_df = create_decade_df(predictions_df)

# 연대별 추이 그래프 그리기
plot_trend(decade_df)


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from adjustText import adjust_text

# 연도별 데이터 프레임 생성 함수
def create_yearly_df(df):
    yearly_df = df.groupby(['year', 'predicted_label']).size().unstack(fill_value=0)
    yearly_df.columns = ['지칭의 저희', '겸양의 저희']
    yearly_df.reset_index(inplace=True)
    yearly_df['차이'] = yearly_df['겸양의 저희'] - yearly_df['지칭의 저희']
    return yearly_df

# 시각화 함수
def plot_difference(yearly_df):
    plt.figure(figsize=(12, 6))

    # 원 데이터 플롯
    plt.plot(yearly_df['year'], yearly_df['차이'], marker='o', linestyle='-', color='g', label='겸양의 저희 - 지칭의 저희')

    # 차이 추세선 추가
    z = np.polyfit(yearly_df['year'], yearly_df['차이'], 1)
    p = np.poly1d(z)
    plt.plot(yearly_df['year'], p(yearly_df['year']), linestyle='--', color='g', alpha=0.5)

    plt.title('연도별 겸양의 저희와 지칭의 저희의 차이')
    plt.xlabel('연도')
    plt.ylabel('차이 (겸양의 저희 - 지칭의 저희)')
    plt.grid(True)

    texts = []
    for i, row in yearly_df.iterrows():
        texts.append(plt.text(row['year'], row['차이'], row['차이'], ha='center', va='bottom', fontsize=9, color='black'))

    adjust_text(texts, arrowprops=dict(arrowstyle="->", color='gray', lw=0.5))

    plt.legend(loc='upper left')
    plt.xticks(yearly_df['year'], rotation=90)
    plt.show()


In [ ]:
# 데이터프레임 확인
print("데이터프레임 확인:")
print(predictions_df)

In [ ]:
# 연도별 데이터 프레임 생성
yearly_df = create_yearly_df(predictions_df)

# 연도별 데이터프레임 확인
print("연도별 데이터프레임 확인:")
yearly_df

In [ ]:
# 연도별 겸양의 저희와 지칭의 저희의 차이 추이 그래프 그리기
plot_difference(yearly_df)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from adjustText import adjust_text

# 연도별 데이터 프레임 생성 함수
def create_yearly_df(df):
    yearly_df = df.groupby(['year', 'predicted_label']).size().unstack(fill_value=0)
    yearly_df.columns = ['지칭의 저희', '겸양의 저희']
    yearly_df.reset_index(inplace=True)
    yearly_df['차이'] = yearly_df['겸양의 저희'] - yearly_df['지칭의 저희']
    return yearly_df

# 시각화 함수
def plot_difference(yearly_df):
    plt.figure(figsize=(14, 8))

    # 원 데이터 플롯
    plt.plot(yearly_df['year'], yearly_df['차이'], marker='o', markersize=10, linestyle='-', linewidth=2, color='darkgreen', label='겸양의 저희 - 지칭의 저희')

    # 차이 추세선 추가
    z = np.polyfit(yearly_df['year'], yearly_df['차이'], 1)
    p = np.poly1d(z)
    plt.plot(yearly_df['year'], p(yearly_df['year']), linestyle='--', linewidth=2, color='orange', alpha=0.7, label='추세선')

    plt.title('연도별 겸양의 저희와 지칭의 저희의 차이', fontsize=16)
    plt.xlabel('연도', fontsize=14)
    plt.ylabel('차이 (겸양의 저희 - 지칭의 저희)', fontsize=14)
    plt.grid(True)

    # 텍스트 레이블 추가
    texts = []
    for i, row in yearly_df.iterrows():
        texts.append(plt.text(row['year'], row['차이'], f'{row["차이"]}', ha='center', va='bottom', fontsize=12, color='black'))

    adjust_text(texts, arrowprops=dict(arrowstyle="->", color='gray', lw=0.5))

    plt.legend(loc='upper left', fontsize=12)
    plt.xticks(yearly_df['year'], rotation=45, fontsize=12)
    plt.yticks(fontsize=12)
    plt.ylim(min(yearly_df['차이']) - 2, max(yearly_df['차이']) + 2)  # y축 범위 조정
    plt.tight_layout()
    plt.show()

# 데이터프레임 확인
print("데이터프레임 확인:")
print(predictions_df)

# 연도별 데이터 프레임 생성
yearly_df = create_yearly_df(predictions_df)

# 연도별 데이터프레임 확인
print("연도별 데이터프레임 확인:")
print(yearly_df)

# 연도별 겸양의 저희와 지칭의 저희의 차이 추이 그래프 그리기
plot_difference(yearly_df)


In [ ]:
predictions_df = pd.read_pickle('/content/drive/Shareddrives/DHKP/2021-2023.기존연구/03. AI융합연구_우리나라_저희(21년_9-12월)/[서재현] 저희/학습용데이터/v3_jeohui_040730_third_data_predictions.pickle')

In [ ]:
predictions_df.columns

In [ ]:
df = predictions_df

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from adjustText import adjust_text

# 연도별 데이터 프레임 생성 함수
def create_yearly_df(df):
    yearly_df = df.groupby(['year', 'predicted_label']).size().unstack(fill_value=0)
    yearly_df.columns = ['지칭의 저희', '겸양의 저희']
    yearly_df.reset_index(inplace=True)
    yearly_df['차이'] = yearly_df['겸양의 저희'] - yearly_df['지칭의 저희']
    return yearly_df

# 시각화 함수
def plot_difference(yearly_df):
    plt.figure(figsize=(14, 8))

    # 막대그래프
    bars = plt.bar(yearly_df['year'], yearly_df['차이'], color=np.where(yearly_df['차이'] >= 0, 'blue', 'red'))

    plt.title('연도별 겸양의 저희와 지칭의 저희의 차이', fontsize=16)
    plt.xlabel('연도', fontsize=14)
    plt.ylabel('차이 (겸양의 저희 - 지칭의 저희)', fontsize=14)
    plt.grid(True, axis='y', linestyle='--', alpha=0.7)

    # 데이터 라벨 추가
    for bar in bars:
        yval = bar.get_height()
        plt.text(bar.get_x() + bar.get_width()/2, yval, int(yval), ha='center', va='bottom' if yval >= 0 else 'top', fontsize=12)

    plt.xticks(yearly_df['year'], rotation=45, fontsize=12)
    plt.yticks(fontsize=12)
    plt.tight_layout()
    plt.show()

    # 연도별 겸양의 저희와 지칭의 저희의 차이 추이 그래프 그리기
plot_difference(yearly_df)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from adjustText import adjust_text

# 연대별 데이터 프레임 생성 함수
def create_decade_df(df):
    df['decade'] = (df['year'] // 10) * 10
    decade_df = df.groupby(['decade', 'predicted_label']).size().unstack(fill_value=0)
    decade_df.columns = ['지칭의 저희', '겸양의 저희']
    decade_df['전체 기사 수'] = decade_df.sum(axis=1)
    decade_df['지칭의 저희 비율'] = decade_df['지칭의 저희'] / decade_df['전체 기사 수']
    decade_df['겸양의 저희 비율'] = decade_df['겸양의 저희'] / decade_df['전체 기사 수']
    decade_df.reset_index(inplace=True)
    return decade_df

# 시각화 함수
def plot_decade_proportions(decade_df):
    plt.figure(figsize=(14, 8))

    # 원 데이터 플롯
    plt.plot(decade_df['decade'], decade_df['겸양의 저희 비율'], marker='o', linestyle='-', linewidth=2, color='blue', label='겸양의 저희 비율')
    plt.plot(decade_df['decade'], decade_df['지칭의 저희 비율'], marker='o', linestyle='-', linewidth=2, color='red', label='지칭의 저희 비율')

    # 겸양의 저희 비율 추세선 추가
    z1 = np.polyfit(decade_df['decade'], decade_df['겸양의 저희 비율'], 1)
    p1 = np.poly1d(z1)
    plt.plot(decade_df['decade'], p1(decade_df['decade']), linestyle='--', linewidth=2, color='blue', alpha=0.5)

    # 지칭의 저희 비율 추세선 추가
    z2 = np.polyfit(decade_df['decade'], decade_df['지칭의 저희 비율'], 1)
    p2 = np.poly1d(z2)
    plt.plot(decade_df['decade'], p2(decade_df['decade']), linestyle='--', linewidth=2, color='red', alpha=0.5)

    plt.title('연대별 전체 기사 수 대비 겸양의 저희와 지칭의 저희 비율', fontsize=16)
    plt.xlabel('연대', fontsize=14)
    plt.ylabel('비율', fontsize=14)
    plt.grid(True)

    texts = []
    for i, row in decade_df.iterrows():
        texts.append(plt.text(row['decade'], row['겸양의 저희 비율'], f'{row["겸양의 저희 비율"]:.2%}', ha='center', va='bottom', fontsize=12, color='blue'))
        texts.append(plt.text(row['decade'], row['지칭의 저희 비율'], f'{row["지칭의 저희 비율"]:.2%}', ha='center', va='top', fontsize=12, color='red'))

    adjust_text(texts, arrowprops=dict(arrowstyle="->", color='gray', lw=0.5))

    plt.legend(loc='upper left', fontsize=12)
    plt.xticks(decade_df['decade'], rotation=45, fontsize=12)
    plt.yticks(fontsize=12)
    plt.tight_layout()
    plt.show()


# 데이터프레임 확인
print("데이터프레임 확인:")
print(predictions_df)

# 연대별 데이터 프레임 생성
decade_df = create_decade_df(predictions_df)

# 연대별 데이터프레임 확인
print("연대별 데이터프레임 확인:")
print(decade_df)

# 연대별 전체 기사 수 대비 겸양의 저희와 지칭의 저희 비율 추이 그래프 그리기
plot_decade_proportions(decade_df)


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# 주어진 데이터를 딕셔너리로 변환
data = {
    'year': [1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999,
             2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009,
             2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019,
             2020, 2021, 2022, 2023, 2024],
    'count': [5820, 5776, 5117, 5163, 5173, 4931, 4912, 5735, 4790, 4659,
              10179, 10582, 9170, 7976, 7702, 6981, 7071, 7651, 7643, 8437,
              8200, 7925, 7619, 7147, 6782, 7071, 6960, 7280, 23759, 28942,
              32976, 24956, 26334, 24675, 14219]
}

# 데이터프레임으로 변환
df = pd.DataFrame(data)

# 시각화
plt.figure(figsize=(14, 8))
plt.plot(df['year'], df['count'], marker='o', linestyle='-', linewidth=2, color='purple', label='우리의 사용 추이')

# 추세선 추가
z = np.polyfit(df['year'], df['count'], 1)
p = np.poly1d(z)
plt.plot(df['year'], p(df['year']), linestyle='--', linewidth=2, color='orange', alpha=0.5, label='추세선')

# 그래프 제목과 축 레이블 추가
plt.title("'우리'의 사용 추이", fontsize=16)
plt.xlabel('연도', fontsize=14)
plt.ylabel('사용 빈도', fontsize=14)
plt.grid(True)

# 데이터 포인트 레이블 추가
for i, row in df.iterrows():
    plt.text(row['year'], row['count'], f'{row["count"]}', ha='center', va='bottom' if row["count"] > 0 else 'top', fontsize=10)

# 범례 추가
plt.legend(loc='upper left', fontsize=12)
plt.xticks(df['year'], rotation=45, fontsize=12)
plt.yticks(fontsize=12)
plt.tight_layout()
plt.show()


In [ ]:
# 연대별 데이터프레임 확인
print("연대별 데이터프레임 확인:")
print(decade_df)

In [ ]:
# 동아일보 연대별 추이 그래프 그리기
plot_trend(decade_df, 'donga')

In [ ]:
# 조선일보 연대별 추이 그래프 그리기
plot_trend(decade_df, 'chosun')

#### 전체

In [ ]:
import os
plt.rc('font', family='NanumBarunGothic')

In [ ]:
df=predictions_df.groupby(['year', 'predicted_label']).count()
df

In [ ]:
df.reset_index(inplace=True)
df

In [ ]:
df.set_index(['year'], inplace=True)
df

In [ ]:
df['겸양의 저희'] =0
df['지칭의 저희'] = 0

for idx in df.index :
    df['겸양의 저희'][idx] = df['text'][df['predicted_label']==1][idx]

    try :
        df['지칭의 저희'][idx] =df['text'][df['predicted_label']==0][idx]
    except :
    #donga['겸양의 저희'][idx] =0
        df['지칭의 저희'][idx] =0
df

In [ ]:
df= df[['겸양의 저희', '지칭의 저희']]

In [ ]:
df.reset_index(inplace=True)

In [ ]:
df= df.drop_duplicates('year')

In [ ]:
df.set_index('year', inplace=True)
df.plot()

In [ ]:
df = predictions_df
df